In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import os, sys
from tqdm import tqdm

sys.path.insert(0, 'src')
from utils import remove_and_create_dir, ensure_dir
from preprocess_data import preprocess_csvs

/n/fs/ac-project/anaconda3/envs/dqn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Process a few data files

In [ ]:
data_root_dir =  os.path.join('data') # TODO: replace with your root if necessary
csv_file_names = ['fattree_rsim1.csv']
csv_paths = [os.path.join(data_root_dir, csv_file_name) for csv_file_name in csv_file_names]
save_dir = os.path.join('data', 'processed_data')

preprocess_csvs(
    csv_paths, 
    verbose=True,
    csv_save_dir=save_dir)

### Process a LOT of data files organized in directories

In [ ]:
data_root_dir =  os.path.join('data') # TODO: replace with your root if necessary
# csv_file_names = ['rsim.csv', 'test.csv', 'train.csv']
sub_dirs = ['itr{}'.format(i) for i in range(1,6)]
csv_file_names = [
    'sch_FIFO-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_DDR-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_WFQ-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_SP-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_FIFO-tgen_OnOff-n_flows_100-n_ports_4.csv',
    'sch_FIFO-tgen_Poisson-n_flows_20-n_ports_4.csv',
    'sch_FIFO-tgen_Poisson-n_flows_60-n_ports_4.csv',
    'sch_FIFO-tgen_Poisson-n_flows_100-n_ports_8.csv',
    'sch_FIFO-tgen_Poisson-n_flows_100-n_ports_16.csv',
]
# print(sub_dirs)

for sub_dir in sub_dirs:
    csv_paths = []
    for csv_file_name in csv_file_names:
        csv_paths.append(os.path.join(data_root_dir, sub_dir, csv_file_name))

    # csv_paths = [os.path.join(data_root_dir, csv_file_name) for csv_file_name in csv_file_names]
    save_dir = os.path.join('data', 'processed_data', sub_dir)
    ensure_dir(save_dir)
    # print(len(csv_paths))
    preprocess_csvs(
        csv_paths, 
        verbose=False,
        csv_save_dir=save_dir)

### Process DQN fattree data files to train on

In [2]:
data_root_dir =  os.path.join('data', 'dqn_data', 'data', 'fattree16', ) # TODO: replace with your root if necessary
# csv_file_names = ['rsim.csv', 'test.csv', 'train.csv']
sub_dirs = ['map', 'onoff', 'poisson']
csv_file_names = [
    'rsim{}.csv'.format(i) for i in range(1,6)
]
# print(sub_dirs)

for sub_dir in sub_dirs:
    csv_paths = []
    for csv_file_name in csv_file_names:
        csv_paths.append(os.path.join(data_root_dir, sub_dir, csv_file_name))
    print(csv_paths)
    # csv_paths = [os.path.join(data_root_dir, csv_file_name) for csv_file_name in csv_file_names]
    save_dir = os.path.join('data', 'processed_data', 'dqn_fattree16', sub_dir)
    ensure_dir(save_dir)
    # print(len(csv_paths))
    preprocess_csvs(
        csv_paths, 
        verbose=False,
        csv_save_dir=save_dir)

['data/dqn_data/data/fattree16/map/rsim1.csv', 'data/dqn_data/data/fattree16/map/rsim2.csv', 'data/dqn_data/data/fattree16/map/rsim3.csv', 'data/dqn_data/data/fattree16/map/rsim4.csv', 'data/dqn_data/data/fattree16/map/rsim5.csv']
Processing 1/5 csv: data/dqn_data/data/fattree16/map/rsim1.csv


100%|█████████████████████████████████████████████| 8/8 [05:22<00:00, 40.37s/it]


Saved processed csv to data/processed_data/dqn_fattree16/map/rsim1_processed.csv

Processing 2/5 csv: data/dqn_data/data/fattree16/map/rsim2.csv


100%|█████████████████████████████████████████████| 8/8 [05:21<00:00, 40.17s/it]


Saved processed csv to data/processed_data/dqn_fattree16/map/rsim2_processed.csv

Processing 3/5 csv: data/dqn_data/data/fattree16/map/rsim3.csv


...

Saved processed csv to data/processed_data/dqn_fattree16/poisson/rsim3_processed.csv

Processing 4/5 csv: data/dqn_data/data/fattree16/poisson/rsim4.csv


100%|█████████████████████████████████████████████| 8/8 [05:19<00:00, 39.90s/it]


Saved processed csv to data/processed_data/dqn_fattree16/poisson/rsim4_processed.csv

Processing 5/5 csv: data/dqn_data/data/fattree16/poisson/rsim5.csv


100%|█████████████████████████████████████████████| 8/8 [05:18<00:00, 39.82s/it]


Saved processed csv to data/processed_data/dqn_fattree16/poisson/rsim5_processed.csv

